In [1]:
import sys
import time
import os
import tensorflow as tf
import numpy as np

from collections import namedtuple
from tensorflow.python import debug as tf_debug
from tensorflow.core.example import example_pb2
from importlib import reload

import json, os, re, shutil, sys, time
import collections, itertools


/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
import glob
import random
import struct
import csv
import batch; reload(batch)
import data; reload(data)
import model; reload(model)

from batch import Example,Batch
from batcher import Batcher
from data import Vocab
from model import SummarizationModel
from decode import BeamSearchDecoder
import training_util as tutil



In [3]:
def setup_decode_params():
    hps_dict = {
        'mode' : 'decode',
        'single_pass' : True,
        'log_root' : '/home/ubuntu/W266/final_0/W266_Final/model_3/saved',
        'cp_dir':'train',
        'hidden_dim' : 200,
        'emb_dim' : 128,
        'batch_size' : 4,     #<---------- Keep same as beam size
        'max_enc_steps' : 400,
        'max_dec_steps' : 1,        #<---------------Keep at 1
        'beam_dec_steps' : 120, # Decode steps during beam search
        'beam_size' : 4,
        'min_dec_steps' : 35,
        'vocab_size' : 50000,
        'lr' : 0.15,
        'adagrad_init_acc' : 0.1,
        'rand_unif_init_mag' : 0.02,
        'trunc_norm_init_std' : 1e-4,
        'max_grad_norm' : 2.0,
        'pointer_gen' : False,
        'coverage' : False,
        'cov_loss_wt' : 1.0,
        'vocab_path' : '/home/ubuntu/W266/final_0/W266_Final/data/final_processed/vocab',
        'data_path' : '/home/ubuntu/W266/final_0/W266_Final/data/final_chunked/validation_001*'
    }
    
    hps = namedtuple("HParams", hps_dict.keys())(**hps_dict)
    return hps

In [4]:
hps = setup_decode_params()
vocab = Vocab(hps.vocab_path, hps.vocab_size)
batches = tutil.get_decode_data(hps,vocab,hps.data_path,False)







max_size of vocab was specified as 50000; we now have 50000 words. Stopping reading.
Finished constructing vocabulary of 50000 total words. Last word added: 16:03
INFO:tensorflow:Fetching data..
INFO:tensorflow:Creating batches..
INFO:tensorflow:[TOTAL Batches]  : 999
INFO:tensorflow:[TOTAL Examples] : 999
INFO:tensorflow:Creating batches..COMPLETE


In [5]:
tf.reset_default_graph()
model_params = dict(hps=hps, 
                    vocab=vocab)

lm = SummarizationModel(**model_params)
lm.BuildCoreGraph()
lm.BuildSamplerGraph()
lm.summarizeGraph()

with lm.graph.as_default():
    initializer = tf.global_variables_initializer()
    saver = tf.train.Saver()

with tf.Session(graph=lm.graph) as session:
    decoder = BeamSearchDecoder(lm, session, vocab, hps, saver)
    decoder.decode(batches) 

INFO:tensorflow:Building core graph...
INFO:tensorflow:Adding attention_decoder timestep 0 of 1
INFO:tensorflow:Building core graph...COMPLETE
INFO:tensorflow:Building Sampler graph...
INFO:tensorflow:Building Sampler graph...COMPLETE
INFO:tensorflow:Loading checkpoint /home/ubuntu/W266/final_0/W266_Final/model_3/saved/train/model-18
INFO:tensorflow:Restoring parameters from /home/ubuntu/W266/final_0/W266_Final/model_3/saved/train/model-18
INFO:tensorflow:Added 0 to file
INFO:tensorflow:Added 1 to file
INFO:tensorflow:Added 2 to file
INFO:tensorflow:Added 3 to file
INFO:tensorflow:Added 4 to file
INFO:tensorflow:Added 5 to file
INFO:tensorflow:Added 6 to file
INFO:tensorflow:Added 7 to file
INFO:tensorflow:Added 8 to file
INFO:tensorflow:Added 9 to file
ROUGE SCORES
{'rouge-1': {'f': 0.0, 'p': 0.0, 'r': 0.0}, 'rouge-2': {'f': 0.0, 'p': 0.0, 'r': 0.0}, 'rouge-l': {'f': 0.0, 'p': 0.0, 'r': 0.0}}
